# Create monthly panel data set

#### merge formatted_final.csv with m_stock_level_data.csv

In [1]:
import pandas as pd

############### import formatted_final
ownership_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final.csv"
df_ownership = pd.read_csv(ownership_path)

############### import m_stock_level_data
path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/m_stock_level_data.csv"
df_add = pd.read_csv(path)

df_add['date'] = pd.to_datetime(df_add['date'])
df_ownership["date"] = pd.to_datetime(df_ownership['date'])

result_df = pd.merge(df_ownership, df_add[['date', 'stock_RIC', 'price', "return1Mo", "gross_profit", "price_to_BV"]], on=['date', 'stock_RIC'], how='left')

############# export as monthly_panel_v1.csv
output_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
result_df.to_csv(output_path, index=False)
display(result_df)

,date,stock_RIC,index_member,stock_value_held,percent_of_traded_shares,FUND_stock_value_held,FUND_percent_of_traded_shares,INDEX_FUND_stock_value_held,INDEX_FUND_percent_of_traded_shares,ACTIVE_FUND_stock_value_held,ACTIVE_FUND_percent_of_traded_shares,market_cap,ETF_ownership,FUND_ownership,INDEX_FUND_ownership,ACTIVE_FUND_ownership,price,return1Mo,gross_profit,price_to_BV
0,2009-12-31,0MW4EUR.xbo^K15,0,10.386378,0.5011,119.213083,5.6151,11.926645,0.6070,107.286438,5.0081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-31,0MW4EUR.xbo^K15,0,10.497117,0.5016,112.354494,5.2454,8.299856,0.4206,104.054639,4.8248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-02-28,0MW4EUR.xbo^K15,0,9.157936,0.4714,136.100369,6.4882,7.903669,0.4086,128.196700,6.0796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-03-31,0MW4EUR.xbo^K15,0,10.744003,0.4934,149.553185,6.7480,7.153383,0.3600,142.399802,6.3880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-30,0MW4EUR.xbo^K15,0,9.726336,0.4578,139.564954,6.2827,14.385590,0.6743,125.179364,5.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,4180.070271,6.3278,19064.195360,28.8973,5710.362177,8.6452,13353.833183,20.2521,6.606391e+10,0.063273,0.288572,0.086437,0.202135,439.079073,-0.964933,NaN,2.772687
196220,2023-08-31,ZURN.S,1,4139.611296,6.5174,18901.923118,29.6819,5687.507826,8.9590,13214.415292,20.7229,6.344198e+10,0.065250,0.297940,0.089649,0.208291,433.477862,-1.378327,NaN,2.907803
196221,2023-09-30,ZURN.S,1,4109.144161,6.4619,18625.671378,29.2231,5646.716146,8.8776,12978.955232,20.3455,6.357877e+10,0.064631,0.292954,0.088814,0.204140,434.412512,1.669894,NaN,2.841227
196222,2023-10-31,ZURN.S,1,4290.094268,6.5556,19011.235820,29.2812,5854.380449,8.9551,13156.855371,20.3261,6.545329e+10,0.065544,0.290455,0.089444,0.201011,447.220479,2.499405,NaN,2.926635


# Variable construction

### calculate monthly standard devation and merge it to monthly panel

In [2]:
import pandas as pd

def load_data(file_path):
    """
    Load stock level data
    """
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.dropna(subset=['date', 'return1D'], inplace=True)
    
    #### return cleaned stock level data
    return df

def calculate_monthly_std_dev(df):
    """
    Take cleaned stock level data to calculate monthly standard deviation and count of returns
    """
    monthly_stats = df.groupby(['stock_RIC', pd.Grouper(key='date', freq='M')])['return1D'].agg(['std', 'count']).reset_index()
    monthly_stats.rename(columns={'std': 'monthly_std_dev', 'count': 'count_returns'}, inplace=True)
    
    return monthly_stats

def merge_stats_with_stock_level(df, monthly_stats):
    """
    Merge monthly standard statistics back onto the monthly_panel data frame
    """

    # Create a year-month identifier for merging
    df['year_month'] = df['date'].dt.to_period('M')
    monthly_stats['year_month'] = monthly_stats['date'].dt.to_period('M')

    # Merge the calculated std dev and count back onto the original DataFrame
    merged_df = pd.merge(df, monthly_stats, on=['stock_RIC', 'year_month'], how='left')

    #Ensure columns are sorted before dropping duplicates
    if 'date_x' in merged_df.columns and 'stock_RIC' in merged_df.columns:
        merged_df.sort_values(by=['stock_RIC', 'date_x'], ascending=[True, False], inplace=True)
        merged_df = merged_df.drop_duplicates(subset=['stock_RIC', 'year_month'], keep='first')

    return merged_df

def clean_and_finalize_data(df):
    """
    Clean and finalize monthly stock level data (with std dev)
    """
    #Ensure columns are sorted before dropping duplicates
    if 'date_x' in df.columns and 'stock_RIC' in df.columns:
        df.sort_values(by=['stock_RIC', 'date_x'], ascending=[True, False], inplace=True)
        df = df.drop_duplicates(subset=['stock_RIC', 'year_month'], keep='first')


    # Drop the helper 'year_month' column if no longer needed
    df.drop(columns=['year_month', 'index'], inplace=True, errors='ignore')

    df['date_y'] = df['date_y'].dt.date
    df['date_y'] = pd.to_datetime(df['date_y'])

    df.rename(columns={'date_y': 'DATE_monthly_std_dev', 'date_x': 'date'}, inplace=True)

    df['date'] = df["date"] - pd.offsets.MonthEnd(1)
    
    return df

def merge_with_panel_data(panel_data_file, df):
    """
    Merge the monthly stock level data (with std dev) to existing monthly panel
    """
    
    df_panel = pd.read_csv(panel_data_file)
    df_panel["date"] = pd.to_datetime(df_panel['date'])
    
    df = df[["stock_RIC", "date", "DATE_monthly_std_dev", "monthly_std_dev", "count_returns"]].copy(deep=True)
    
    merged_df = pd.merge(df_panel, df, on=['date', 'stock_RIC'], how='left')
    merged_df.to_csv(panel_data_file, index=False)
    return merged_df


stock_level_file = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv"
panel_data_file = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"

df_clean = load_data(stock_level_file)
monthly_std_dev = calculate_monthly_std_dev(df_clean)
df_stock_level = merge_stats_with_stock_level(df_clean, monthly_std_dev)
display(df_stock_level)
df_stock_level = clean_and_finalize_data(df_stock_level)
display(df_stock_level)
df_merged = merge_with_panel_data(panel_data_file, df_stock_level)

display(df_merged)

,index,stock_RIC,date_x,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price,year_month,date_y,monthly_std_dev,count_returns
1185542,1185671,0MW4EUR.xbo^K15,2015-11-03,14.970000,-0.133422,0.133779,-0.066756,938065.0,1.404420e+07,7.302660e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11,2015-11-30,0.008801,2076
1165736,1165858,0MW4EUR.xbo^K15,2015-10-30,14.950000,-0.066845,-0.333333,-0.133601,4538663.0,6.787420e+07,7.291920e+09,2.235776e+09,NaN,14.650000,15.260000,2015-10,2015-10-31,0.149910,22
1145989,1146111,0MW4EUR.xbo^K15,2015-09-30,14.970000,0.066845,-0.200000,0.335121,7431477.0,1.112457e+08,7.305110e+09,2.235776e+09,NaN,14.930000,14.980000,2015-09,2015-09-30,0.201650,22
1127377,1127499,0MW4EUR.xbo^K15,2015-08-31,14.940000,0.134048,0.403226,-1.059603,3412482.0,5.096370e+07,7.290470e+09,2.235776e+09,NaN,14.870000,15.000000,2015-08,2015-08-31,0.498490,21
1106891,1107012,0MW4EUR.xbo^K15,2015-07-31,15.100000,0.599600,-0.132275,-0.264201,1315684.0,1.978780e+07,7.367691e+09,2.235776e+09,NaN,14.660000,15.410000,2015-07,2015-07-31,0.483467,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78981,78983,ZURN.S,2010-05-31,166.645847,0.042248,3.001305,-1.415487,286368.0,4.778093e+07,2.468892e+10,NaN,1.053197,166.645847,166.786595,2010-05,2010-05-31,2.251636,19
63612,63613,ZURN.S,2010-04-30,167.705219,-1.678264,-3.417772,-5.938007,1179841.0,1.999244e+08,2.474560e+10,NaN,1.100166,167.705219,167.844857,2010-04,2010-04-30,1.236923,20
48149,48150,ZURN.S,2010-03-31,189.913011,-0.331858,1.235955,4.362934,1636985.0,3.115723e+08,2.802245e+10,NaN,1.265999,189.842751,189.913011,2010-03,2010-03-31,0.622809,23
30553,30553,ZURN.S,2010-02-26,177.091914,1.092896,1.688261,13.199301,876093.0,1.546427e+08,2.612145e+10,NaN,1.219970,177.023538,177.091914,2010-02,2010-02-28,1.339416,20


,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price,DATE_monthly_std_dev,monthly_std_dev,count_returns
1185542,0MW4EUR.xbo^K15,2015-10-31,14.970000,-0.133422,0.133779,-0.066756,938065.0,1.404420e+07,7.302660e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11-30,0.008801,2076
1165736,0MW4EUR.xbo^K15,2015-09-30,14.950000,-0.066845,-0.333333,-0.133601,4538663.0,6.787420e+07,7.291920e+09,2.235776e+09,NaN,14.650000,15.260000,2015-10-31,0.149910,22
1145989,0MW4EUR.xbo^K15,2015-08-31,14.970000,0.066845,-0.200000,0.335121,7431477.0,1.112457e+08,7.305110e+09,2.235776e+09,NaN,14.930000,14.980000,2015-09-30,0.201650,22
1127377,0MW4EUR.xbo^K15,2015-07-31,14.940000,0.134048,0.403226,-1.059603,3412482.0,5.096370e+07,7.290470e+09,2.235776e+09,NaN,14.870000,15.000000,2015-08-31,0.498490,21
1106891,0MW4EUR.xbo^K15,2015-06-30,15.100000,0.599600,-0.132275,-0.264201,1315684.0,1.978780e+07,7.367691e+09,2.235776e+09,NaN,14.660000,15.410000,2015-07-31,0.483467,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78981,ZURN.S,2010-04-30,166.645847,0.042248,3.001305,-1.415487,286368.0,4.778093e+07,2.468892e+10,NaN,1.053197,166.645847,166.786595,2010-05-31,2.251636,19
63612,ZURN.S,2010-03-31,167.705219,-1.678264,-3.417772,-5.938007,1179841.0,1.999244e+08,2.474560e+10,NaN,1.100166,167.705219,167.844857,2010-04-30,1.236923,20
48149,ZURN.S,2010-02-28,189.913011,-0.331858,1.235955,4.362934,1636985.0,3.115723e+08,2.802245e+10,NaN,1.265999,189.842751,189.913011,2010-03-31,0.622809,23
30553,ZURN.S,2010-01-31,177.091914,1.092896,1.688261,13.199301,876093.0,1.546427e+08,2.612145e+10,NaN,1.219970,177.023538,177.091914,2010-02-28,1.339416,20


,date,stock_RIC,index_member,stock_value_held,percent_of_traded_shares,FUND_stock_value_held,FUND_percent_of_traded_shares,INDEX_FUND_stock_value_held,INDEX_FUND_percent_of_traded_shares,ACTIVE_FUND_stock_value_held,...,FUND_ownership,INDEX_FUND_ownership,ACTIVE_FUND_ownership,price,return1Mo,gross_profit,price_to_BV,DATE_monthly_std_dev,monthly_std_dev,count_returns
0,2009-12-31,0MW4EUR.xbo^K15,0,10.386378,0.5011,119.213083,5.6151,11.926645,0.6070,107.286438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,2010-01-31,0MW4EUR.xbo^K15,0,10.497117,0.5016,112.354494,5.2454,8.299856,0.4206,104.054639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,2010-02-28,0MW4EUR.xbo^K15,0,9.157936,0.4714,136.100369,6.4882,7.903669,0.4086,128.196700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,2010-03-31,0MW4EUR.xbo^K15,0,10.744003,0.4934,149.553185,6.7480,7.153383,0.3600,142.399802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,2010-04-30,0MW4EUR.xbo^K15,0,9.726336,0.4578,139.564954,6.2827,14.385590,0.6743,125.179364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,4180.070271,6.3278,19064.195360,28.8973,5710.362177,8.6452,13353.833183,...,0.288572,0.086437,0.202135,439.079073,-0.964933,NaN,2.772687,2023-08-31,0.727499,22.0
196220,2023-08-31,ZURN.S,1,4139.611296,6.5174,18901.923118,29.6819,5687.507826,8.9590,13214.415292,...,0.297940,0.089649,0.208291,433.477862,-1.378327,NaN,2.907803,2023-09-30,0.588205,21.0
196221,2023-09-30,ZURN.S,1,4109.144161,6.4619,18625.671378,29.2231,5646.716146,8.8776,12978.955232,...,0.292954,0.088814,0.204140,434.412512,1.669894,NaN,2.841227,2023-10-31,0.910886,22.0
196222,2023-10-31,ZURN.S,1,4290.094268,6.5556,19011.235820,29.2812,5854.380449,8.9551,13156.855371,...,0.290455,0.089444,0.201011,447.220479,2.499405,NaN,2.926635,2023-11-30,0.619786,22.0


### Amihud ratio

In [3]:
import pandas as pd

stock_level_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv"
stock_level_df = pd.read_csv(stock_level_path)

panel_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
panel_df = pd.read_csv(panel_path)

#####################################################
stock_level_df['date'] = pd.to_datetime(stock_level_df['date'])
panel_df['date'] = pd.to_datetime(panel_df['date'])

# Calculate the Amihud ratio
stock_level_df['amihud_ratio'] = abs(stock_level_df['return1D']) / stock_level_df['volume']

# Group by stock and month, and calculate the mean Amihud ratio for each month
stock_level_df['month'] = stock_level_df['date'].dt.to_period('M')
monthly_amihud = stock_level_df.groupby(['stock_RIC', 'month']).agg({
    'amihud_ratio': 'mean',
    'date': 'max'  # Takes the last available date in the month, which might not be the last day
}).reset_index()

# Set each date to the last day of the respective month
monthly_amihud['date'] = monthly_amihud['month'].dt.to_timestamp(how='end').dt.date

# Drop the 'month' column as it's no longer needed
monthly_amihud.drop('month', axis=1, inplace=True)
monthly_amihud['date'] = pd.to_datetime(monthly_amihud['date'])

######### merge this shit
merged_df = pd.merge(panel_df, monthly_amihud, on=['date', 'stock_RIC'], how='left')
display(merged_df)

# Save the merged DataFrame
merged_df.to_csv(panel_path, index = False)

,date,stock_RIC,index_member,stock_value_held,percent_of_traded_shares,FUND_stock_value_held,FUND_percent_of_traded_shares,INDEX_FUND_stock_value_held,INDEX_FUND_percent_of_traded_shares,ACTIVE_FUND_stock_value_held,...,INDEX_FUND_ownership,ACTIVE_FUND_ownership,price,return1Mo,gross_profit,price_to_BV,DATE_monthly_std_dev,monthly_std_dev,count_returns,amihud_ratio
0,2009-12-31,0MW4EUR.xbo^K15,0,10.386378,0.5011,119.213083,5.6151,11.926645,0.6070,107.286438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-31,0MW4EUR.xbo^K15,0,10.497117,0.5016,112.354494,5.2454,8.299856,0.4206,104.054639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-02-28,0MW4EUR.xbo^K15,0,9.157936,0.4714,136.100369,6.4882,7.903669,0.4086,128.196700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-03-31,0MW4EUR.xbo^K15,0,10.744003,0.4934,149.553185,6.7480,7.153383,0.3600,142.399802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-30,0MW4EUR.xbo^K15,0,9.726336,0.4578,139.564954,6.2827,14.385590,0.6743,125.179364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,4180.070271,6.3278,19064.195360,28.8973,5710.362177,8.6452,13353.833183,...,0.086437,0.202135,439.079073,-0.964933,NaN,2.772687,2023-08-31,0.727499,22.0,0.000003
196220,2023-08-31,ZURN.S,1,4139.611296,6.5174,18901.923118,29.6819,5687.507826,8.9590,13214.415292,...,0.089649,0.208291,433.477862,-1.378327,NaN,2.907803,2023-09-30,0.588205,21.0,0.000003
196221,2023-09-30,ZURN.S,1,4109.144161,6.4619,18625.671378,29.2231,5646.716146,8.8776,12978.955232,...,0.088814,0.204140,434.412512,1.669894,NaN,2.841227,2023-10-31,0.910886,22.0,0.000002
196222,2023-10-31,ZURN.S,1,4290.094268,6.5556,19011.235820,29.2812,5854.380449,8.9551,13156.855371,...,0.089444,0.201011,447.220479,2.499405,NaN,2.926635,2023-11-30,0.619786,22.0,0.000003


### past 12-month returns

In [4]:
import pandas as pd

panel_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
df = pd.read_csv(panel_path)

# Convert 'date' to datetime format and ensure it is sorted
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['stock_RIC', 'date'], inplace=True)

# Calculate the price 12 months ago by shifting the prices within each stock group
df['price_12_months_ago'] = df.groupby('stock_RIC')['price'].shift(12)

# Calculate the 12-month cumulative return
df['cumulative_return_12m'] = (df['price'] - df['price_12_months_ago']) / df['price_12_months_ago']

# Drop the helper column if it's no longer needed
df.drop('price_12_months_ago', axis=1, inplace=True)
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

df.to_csv(panel_path, index=False)

### bid ask spread

In [5]:
import pandas as pd

def calculate_monthly_bid_ask_spread():

    ################################## import stock_level_data
    stock_level_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv"
    df = pd.read_csv(stock_level_path)

    df = df[["stock_RIC", "date", "bid_price", "ask_price"]].copy(deep = True)
    # Ensure the date column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Calculate the daily bid-ask spread and the midpoint
    df['bid_ask_spread'] = df['ask_price'] - df['bid_price']
    df['midpoint'] = (df['ask_price'] + df['bid_price']) / 2
    df['relative_spread'] = df['bid_ask_spread'] / df['midpoint']

    # Set date as index
    df.set_index('date', inplace=True)

    # Calculate the monthly average of the relative spread
    monthly_spread = df.groupby([pd.Grouper(freq='M'), 'stock_RIC'])['relative_spread'].mean().reset_index()

    # Rename columns for clarity
    monthly_spread.rename(columns={'relative_spread': 'monthly_relative_spread'}, inplace=True)


    ################################## import monthly_panel
    panel_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
    panel_df = pd.read_csv(panel_path)
    panel_df['date'] = pd.to_datetime(panel_df['date'])


    ################################## merge monthly_spread with monthly_panel
    monthly_spread['year_month'] = monthly_spread['date'].dt.to_period('M')
    panel_df['year_month'] = panel_df['date'].dt.to_period('M')
    monthly_spread = monthly_spread.drop(columns=['date'])

    merged_df = pd.merge(panel_df, monthly_spread, on = ["stock_RIC", "year_month"], how = "left")
    merged_df.to_csv(panel_path, index=False)
    display(merged_df)

calculate_monthly_bid_ask_spread()

,date,stock_RIC,index_member,stock_value_held,percent_of_traded_shares,FUND_stock_value_held,FUND_percent_of_traded_shares,INDEX_FUND_stock_value_held,INDEX_FUND_percent_of_traded_shares,ACTIVE_FUND_stock_value_held,...,return1Mo,gross_profit,price_to_BV,DATE_monthly_std_dev,monthly_std_dev,count_returns,amihud_ratio,cumulative_return_12m,year_month,monthly_relative_spread
0,2009-12-31,0MW4EUR.xbo^K15,0,10.386378,0.5011,119.213083,5.6151,11.926645,0.6070,107.286438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-12,NaN
1,2010-01-31,0MW4EUR.xbo^K15,0,10.497117,0.5016,112.354494,5.2454,8.299856,0.4206,104.054639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-01,NaN
2,2010-02-28,0MW4EUR.xbo^K15,0,9.157936,0.4714,136.100369,6.4882,7.903669,0.4086,128.196700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-02,NaN
3,2010-03-31,0MW4EUR.xbo^K15,0,10.744003,0.4934,149.553185,6.7480,7.153383,0.3600,142.399802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03,NaN
4,2010-04-30,0MW4EUR.xbo^K15,0,9.726336,0.4578,139.564954,6.2827,14.385590,0.6743,125.179364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-04,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,4180.070271,6.3278,19064.195360,28.8973,5710.362177,8.6452,13353.833183,...,-0.964933,NaN,2.772687,2023-08-31,0.727499,22.0,0.000003,0.028789,2023-07,0.000240
196220,2023-08-31,ZURN.S,1,4139.611296,6.5174,18901.923118,29.6819,5687.507826,8.9590,13214.415292,...,-1.378327,NaN,2.907803,2023-09-30,0.588205,21.0,0.000003,-0.018435,2023-08,0.000242
196221,2023-09-30,ZURN.S,1,4109.144161,6.4619,18625.671378,29.2231,5646.716146,8.8776,12978.955232,...,1.669894,NaN,2.841227,2023-10-31,0.910886,22.0,0.000002,0.063450,2023-09,0.000238
196222,2023-10-31,ZURN.S,1,4290.094268,6.5556,19011.235820,29.2812,5854.380449,8.9551,13156.855371,...,2.499405,NaN,2.926635,2023-11-30,0.619786,22.0,0.000003,0.035510,2023-10,0.000247


### lagged monthly standard deviation

In [6]:
import pandas as pd

def add_lagged_monthly_std_dev():
    path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
    df = pd.read_csv(path, index_col=False)
    # Ensure the date column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Sort the DataFrame by stock_RIC and date
    df = df.sort_values(by=['stock_RIC', 'date'])

    # Group by stock_RIC and shift the monthly_std_dev column by one period to get the lagged value
    df['T1_monthly_std_dev'] = df.groupby('stock_RIC')['monthly_std_dev'].shift(1)
    df['T2_monthly_std_dev'] = df.groupby('stock_RIC')['monthly_std_dev'].shift(2)
    df['T3_monthly_std_dev'] = df.groupby('stock_RIC')['monthly_std_dev'].shift(3)

    df.to_csv(path, index=False)

add_lagged_monthly_std_dev()

# Add country and industry fixed effects

### merge country and industry variables to quarterly data

In [2]:
import pandas as pd

df_q = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv", index_col=False)

df_fixed_vars = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data_countries.csv", index_col=False)

merged_df = pd.merge(df_q, df_fixed_vars, on='stock_RIC', how='left')
display(merged_df)

merged_df.to_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv", index=False)

,date,stock_RIC,index_member,stock_value_held,percent_of_traded_shares,FUND_stock_value_held,FUND_percent_of_traded_shares,INDEX_FUND_stock_value_held,INDEX_FUND_percent_of_traded_shares,ACTIVE_FUND_stock_value_held,...,cumulative_return_12m,year_month,monthly_relative_spread,T1_monthly_std_dev,T2_monthly_std_dev,T3_monthly_std_dev,headquarters_country,exchange_country,business_sector,economic_sector
0,2009-12-31,0MW4EUR.xbo^K15,0,10.386378,0.5011,119.213083,5.6151,11.926645,0.6070,107.286438,...,NaN,2009-12,NaN,NaN,NaN,NaN,Italy,Italy,Automobiles & Auto Parts,Consumer Cyclicals
1,2010-01-31,0MW4EUR.xbo^K15,0,10.497117,0.5016,112.354494,5.2454,8.299856,0.4206,104.054639,...,NaN,2010-01,NaN,NaN,NaN,NaN,Italy,Italy,Automobiles & Auto Parts,Consumer Cyclicals
2,2010-02-28,0MW4EUR.xbo^K15,0,9.157936,0.4714,136.100369,6.4882,7.903669,0.4086,128.196700,...,NaN,2010-02,NaN,NaN,NaN,NaN,Italy,Italy,Automobiles & Auto Parts,Consumer Cyclicals
3,2010-03-31,0MW4EUR.xbo^K15,0,10.744003,0.4934,149.553185,6.7480,7.153383,0.3600,142.399802,...,NaN,2010-03,NaN,NaN,NaN,NaN,Italy,Italy,Automobiles & Auto Parts,Consumer Cyclicals
4,2010-04-30,0MW4EUR.xbo^K15,0,9.726336,0.4578,139.564954,6.2827,14.385590,0.6743,125.179364,...,NaN,2010-04,NaN,NaN,NaN,NaN,Italy,Italy,Automobiles & Auto Parts,Consumer Cyclicals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,4180.070271,6.3278,19064.195360,28.8973,5710.362177,8.6452,13353.833183,...,0.028789,2023-07,0.000240,0.821159,0.555470,1.162274,Switzerland,Switzerland,Insurance,Financials
196220,2023-08-31,ZURN.S,1,4139.611296,6.5174,18901.923118,29.6819,5687.507826,8.9590,13214.415292,...,-0.018435,2023-08,0.000242,0.727499,0.821159,0.555470,Switzerland,Switzerland,Insurance,Financials
196221,2023-09-30,ZURN.S,1,4109.144161,6.4619,18625.671378,29.2231,5646.716146,8.8776,12978.955232,...,0.063450,2023-09,0.000238,0.588205,0.727499,0.821159,Switzerland,Switzerland,Insurance,Financials
196222,2023-10-31,ZURN.S,1,4290.094268,6.5556,19011.235820,29.2812,5854.380449,8.9551,13156.855371,...,0.035510,2023-10,0.000247,0.910886,0.588205,0.727499,Switzerland,Switzerland,Insurance,Financials


### old code

In [7]:
# df_dependent = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")
# df_dependent['date'] = pd.to_datetime(df_dependent['date'], errors='coerce')
# df_dependent.dropna(subset=['date'], inplace=True)
# df_dependent.dropna(subset=['return1D'], inplace=True)

# # Group by 'stock_RIC' and month, then calculate standard deviation and count of 'returns'
# monthly_stats = df_dependent.groupby(['stock_RIC', pd.Grouper(key='date', freq='M')])['return1D'].agg(['std', 'count']).reset_index()
# monthly_stats.rename(columns={'std': 'monthly_std_dev', 'count': 'count_returns'}, inplace=True)

# # Create a year-month identifier for merging
# df_dependent['year_month'] = df_dependent['date'].dt.to_period('M')
# monthly_stats['year_month'] = monthly_stats['date'].dt.to_period('M')

# # Merge the calculated std dev and count back onto the original DataFrame
# df_dependent = pd.merge(df_dependent, monthly_stats, on=['stock_RIC', 'year_month'], how='left')

# #Ensure columns are sorted before dropping duplicates
# if 'date_x' in df_dependent.columns and 'stock_RIC' in df_dependent.columns:
#     df_dependent.sort_values(by=['stock_RIC', 'date_x'], ascending=[True, False], inplace=True)
#     df_dependent = df_dependent.drop_duplicates(subset=['stock_RIC', 'year_month'], keep='first')


# # Drop the helper 'year_month' column if no longer needed
# df_dependent.drop('year_month', axis=1, inplace=True)
# df_dependent.drop('index', axis=1, inplace=True)

# df_dependent['date_y'] = df_dependent['date_y'].dt.date
# df_dependent['date_y'] = pd.to_datetime(df_dependent['date_y'])

# df_dependent.rename(columns={'date_y': 'DATE_monthly_std_dev'}, inplace=True)
# df_dependent.rename(columns={'date_x': 'date'}, inplace=True)

# df_dependent['date'] = df_dependent["date"] - pd.offsets.MonthEnd(1)

# df_dependent.to_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/monthly_st_dev.csv")




# #################### merge dependent variable to monthly_panel

# # Assume df_dependent is defined earlier in your script
# df_dependent_sub = df_dependent[["stock_RIC", "date", "DATE_monthly_std_dev", "monthly_std_dev", "count_returns"]].copy(deep=True)

# display(df_dependent_sub)

# # # Save the result to CSV
# df_dependent_sub.to_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/monthly_st_dev.csv", index=False)

# output_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
# df_monthly = pd.read_csv(output_path)
# df_monthly["date"] = pd.to_datetime(df_monthly['date'])
# df_monthly["date"] = pd.to_datetime(df_monthly['date'])
# display(df_monthly)

# # Merge DataFrames
# merged_df = pd.merge(df_monthly, df_dependent_sub, on=['date', 'stock_RIC'], how='left')
# display(merged_df)

# # Save the merged DataFrame
# merged_df.to_csv(output_path, index = False)